In [1]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import streamlit as st
from langchain_community.document_loaders import CSVLoader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage, ToolMessage
from langchain.output_parsers.openai_tools import (
    PydanticToolsParser,
)
from typing import List
from langchain_core.pydantic_v1 import BaseModel, Field, ValidationError
from langsmith import traceable
from langchain_core.tools import tool
from typing import List
import json
from langgraph.graph import END, MessageGraph
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.prebuilt.tool_executor import ToolInvocation, ToolExecutor
from langchain.output_parsers.openai_tools import JsonOutputToolsParser
from collections import defaultdict
from langchain_core.tools import tool
from typing import List
import os

In [2]:

embedding_function = OpenAIEmbeddings()

loader = CSVLoader("./raw_data.csv", encoding="windows-1252")
documents = loader.load()

db = Chroma.from_documents(documents, embedding_function)
retriever = db.as_retriever()

In [3]:

actor_prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
                You are an engaging and dynamic community of YouTube enthusiasts, named Disney Gamers and Park Lovers, with a shared passion for a wide array of interests that range from the magical realms of Disney lifestyle and content, the creative world of indie gaming, the art of cosplay and fabrication, to the ever-evolving landscape of gaming and tech news. Your community thrives on exploring the wonders of Disney parks and travel, celebrating fan art and comic illustrations, diving deep into the genres of fantasy TV and horror fandoms, and pursuing personal development through hobbies like LEGOs and toy photography.

                **Openness to Change:** Your community values life full of excitement, novelties, and challenges, embracing the freedom to be creative and to determine your actions. With a strong interest in immersion, exploration, and creativity in video games, you seek experiences that allow you to feel like a kid again, engage with beloved characters, and feel a sense of nostalgia. Your diverse age group predominantly falls within the 25-34 range, reflecting a youthful spirit eager for adventure, whether it’s through the vast universes of role-playing games or the imaginative world of indie gaming.

                **Self-enhancement:** Driven by the psychological drivers of proving competence/skills, expressing individuality, and living an exciting life, you strive for recognition from your peers and respect from others. You indulge in hobbies and habits that foster creativity, such as art/photography, arts and crafts, and enjoying museum/performing arts, showcasing a community that values self-expression and personal growth.

                **Conservation:** Although you cherish freedom and creativity, there’s a significant portion of the community that values safety in oneself and family, and avoiding upsetting or harming people. The fact that a considerable number of you value routine and schedules, keep work and life separate, and exercise regularly, shows a commitment to maintaining balance and well-being in your lives. Your community respects the importance of family, as seen through regular family meals and the sentiment that family time is the best part of the day.

                **Self-transcendence:** Your community is not just about self-interest; it also values everyone being treated equally, caring for nature, and acceptance of those who are different. These values reflect a collective commitment to making the world a better place, not just for yourselves but for others as well. Your engagement in video games that offer a sense of community and your interest in genres that promote exploration and adventure speak to your desire to transcend your own experiences and connect with something greater.

                Together, you form a vibrant tapestry of gamers, creators, and dreamers, united by your love for storytelling, creativity, and the pursuit of excitement. Here, every voice matters, every passion is celebrated, and every day is an opportunity to explore new horizons and create memorable experiences. Welcome to a community where dreams come alive, and the magic never ends.

                1. Provide a detailed answer to the user's question.
                    - You MUST include numerical citations in your revised answer to ensure it can be verified.
                    - Add a "References" section to the bottom of your answer. In form of:
                        - [1] source: raw_data.csv (line 1) 
                        - [2] source: refined_data.csv (line 25)
                2. Reflect and critique your answer. Be severe to maximize improvement.
                3. Recommend queries to a statistical data aimed at enhancing your response by identifying significant patterns and deepening your understanding of this community. These queries should specifically target insights from psychology, marketing, or social media.
            """
        ),
        MessagesPlaceholder(variable_name="messages"),
        ("system", "Answer the user's question above using the required format and context.\n {context}")
    ]
)

class Reflection(BaseModel):
    missing: str = Field(description="Critique of what is missing to give insights about the community behavior.")
    superfluous: str = Field(description="Critique of what is superfluous.")

class Answer(BaseModel):
    answer: str = Field(description="A detailed answer to the question.")
    references: List[str] = Field(description="Citations motivating your answer.")

class AnswerQuestion(BaseModel):
    """Answer the question."""

    answer: Answer = Field(description="A detailed answer to the question with references.")
    # reflection: Reflection = Field(description="Your reflection on the initial answer.")
    # search_queries: List[str] = Field(
    #     description="1-3 queries to a statistical data aimed at enhancing your response by identifying significant patterns and deepening your understanding of this community. These queries should specifically target insights from psychology, marketing, or social media."
    # )
    
llm = ChatOpenAI(model="gpt-4-turbo-preview")

from langchain.chains import LLMCheckerChain
# checker_chain = LLMCheckerChain.from_llm(llm)
# initial_answer_chain = actor_prompt_template | checker_chain.bind(tools=[AnswerQuestion], tool_choice="AnswerQuestion")
initial_answer_chain = actor_prompt_template | llm.bind_tools(tools=[AnswerQuestion], tool_choice="AnswerQuestion")

validator = PydanticToolsParser(tools=[AnswerQuestion])

class ResponderWithRetries:
    def __init__(self, runnable, validator, retriever):
        self.runnable = runnable
        self.validator = validator
        self.retriever = retriever

    @traceable
    def respond(self, state: List[BaseMessage]):
        response = []
        context = self.retriever.get_relevant_documents(state[-1].content)
        for attempt in range(3):
            try:
                response = self.runnable.invoke({"messages": state, "context": context})
                print(response)
                self.validator.invoke(response)
                return {
                    "answer": json.loads(response.additional_kwargs['tool_calls'][0]['function']['arguments'])['answer']['answer'],
                    # "answer": response['result'],
                    "contexts": context
                }
            except ValidationError as e:
                state = state + [HumanMessage(content=repr(e))]
        return {
            "answer": json.loads(response.additional_kwargs['tool_calls'][0]['function']['arguments'])['answer']['answer'],
            # "answer": response['result'],
            "contexts": context
        }

first_responder = ResponderWithRetries(
    runnable=initial_answer_chain, validator=validator, retriever=retriever
)

parser = JsonOutputToolsParser(return_id=True)

In [ ]:
res = first_responder.respond([HumanMessage(content="What are the key values of the Disney Gamers and Park Lovers community?")])

In [4]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = 'ls__ac90e0c9b24241de8ffef73129fc3ab1'

In [6]:
from langsmith import Client 

# QA
inputs = [
    "What are the top interests of the Disney Gamers and Park Travelers community?",
    "What is the top hobby of this audience and what percentage of them play video games for more than 5 hours per week?",
    "What is the top interest of the community “Disney Gamers and Park Travelers”?",
    "What are the top hobbies and habits of the Disney Gamers and Park Travelers audience?",
    "What is the top interest of the Disney Gamers and Park Travelers audience?"
]

outputs = [
    "The top interests of the Disney Gamers and Park Travelers community include Disney lifestyle and content, indie gaming, cosplay content and fabrication, gaming and tech news, Disney parks and travel, fan art, comic illustrations, fantasy TV, horror fandoms, personal development, LEGOs, and toy photography.",
    "The top hobby of this audience is Disneyland travel/vacation, with 55.12% of them participating in it. 93.75% of this audience plays video games, with 26.05% of them playing for 5-10 hours per week.",
    "The top interest of the community “Disney Gamers and Park Travelers” is Disney Lifestyle & Content.",
    "According to the document, the top hobbies and habits of the Disney Gamers and Park Travelers audience are entertainment such as going to movies (45.87%), travel/vacation such as going to theme parks (55.12%), and gaming such as arcade games (37.39%).",
    "The top interest of the Disney Gamers and Park Travelers audience is “Disney Lifestyle & Content."
]

qa_pairs = [{"question": q, "answer": a} for q, a in zip(inputs, outputs)]

# Create dataset
client = Client()
dataset_name = "RAG_test_LCEL"
dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="QA pairs about LCEL.",
)
client.create_examples(
    inputs=[{"question": q} for q in inputs],
    outputs=[{"answer": a} for a in outputs],
    dataset_id=dataset.id,
)

In [7]:
def predict_rag_answer(example: dict):
    """Use this for answer evaluation"""
    response = first_responder.respond([HumanMessage(content=example["question"])])
    return {"answer": response["answer"]}

def predict_rag_answer_with_context(example: dict):
    """Use this for evaluation of retrieved documents and hallucinations"""
    response = first_responder.respond(example["question"])
    return {"answer": response["answer"], "contexts": response["contexts"]}

In [8]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

# Evaluator 
qa_evalulator = [LangChainStringEvaluator("cot_qa",     
                                          prepare_data=lambda run, example: {
                                              "prediction": run.outputs["answer"], 
                                              "reference": example.outputs["answer"],
                                              "input": example.inputs["question"],
                                          }  
                                         )]
dataset_name = "RAG_test_LCEL"
experiment_results = evaluate(
    predict_rag_answer,
    data=dataset_name,
    evaluators=qa_evalulator,
    experiment_prefix="rag-qa-oai",
    metadata={"variant": "LCEL context, gpt-3.5-turbo"},
)

/home/vscode/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


View the evaluation results for experiment: 'rag-qa-oai-7a60b14a' at:
https://smith.langchain.com/o/bf63190d-7360-5697-91b9-93341b792d07/datasets/1a4ce892-88ac-4c85-8c2f-2363b0cab28b/compare?selectedSessions=f0fbcd25-aec3-462a-a2a9-5185063e6d77




0it [00:00, ?it/s]

content='' additional_kwargs={'tool_calls': [{'id': 'call_8ZFDnwnN33Xyy6UHhoYkmRGw', 'function': {'arguments': '{"answer":{"answer":"The top interest of the community “Disney Gamers and Park Travelers” is the exploration and enjoyment of Disney parks and attractions. This interest is deeply rooted in their love for the immersive experiences that Disney parks offer, allowing them to connect with their favorite characters and stories in a tangible way. The community’s enthusiasm extends to both the physical experiences at the parks and the virtual exploration through video games that feature Disney themes or characters. This dual interest highlights their passion for all things Disney, from the thrill of the rides to the nostalgia and joy of interacting with the characters in a gaming environment.","references":[]}}', 'name': 'AnswerQuestion'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 126, 'prompt_tokens': 860, 'total_tokens': 986}, 'model_name': 'gpt

5it [00:27,  5.53s/it]


In [ ]:
from langchain.chains import LLMCheckerChain
from langchain_openai import OpenAI

llm = OpenAI(temperature=0.7)

text = "What type of mammal lays the biggest eggs?"

checker_chain = LLMCheckerChain.from_llm(llm, verbose=True)

checker_chain.invoke(text)